#### **Import Modules**

In [5]:
import os
import csv
import json
import time
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
from datetime import datetime



In [ ]:

base_dir = r"D:\CourseworkFolder\DPSynthData\Data Manipulation\mimic-iv-3.1\extracted"

COLUMN_MAP = {
    "transfers.csv": ["subject_id", "hadm_id", "intime", "careunit"],
    "admissions.csv": ["subject_id", "hadm_id", "admittime", "dischtime", "admission_location", "discharge_location"],
    "icustays.csv": ["subject_id", "hadm_id", "intime", "outtime", "first_careunit", "last_careunit"],
    "labevents.csv": ["subject_id", "hadm_id", "charttime", "itemid", "valuenum", "valueuom"],
    "diagnoses_icd.csv": ["subject_id", "hadm_id", "icd_code"],
    "chartevents.csv": ["subject_id", "hadm_id", "charttime", "itemid"],
    "procedureevents.csv": ["subject_id", "hadm_id", "starttime", "itemid"],
    "inputevents.csv": ["subject_id", "hadm_id", "starttime", "itemid"],
    "procedures_icd.csv": ["subject_id", "hadm_id", "chartdate", "icd_code"],
    "hcpcsevents.csv": ["subject_id", "hadm_id", "chartdate", "hcpcs_cd"],
    "poe.csv": ["subject_id", "hadm_id", "ordertime", "poe_id", "order_type", "order_subtype"],
}


FILE_MERGE_MAP = {
    "d_icd_diagnoses.csv": {"key": "icd_code", "merge_file": "diagnoses_icd.csv", "merge_column": "long_title"},
    "d_icd_procedures.csv": {"key": "icd_code", "merge_file": "procedures_icd.csv", "merge_column": "long_title"},
    "d_labitems.csv": {"key": "itemid", "merge_file": "labevents.csv", "merge_column": ["label", "category"]},
    "d_hcpcs.csv": {"key": "code", "merge_file": "hcpcsevents.csv", "merge_column": "short_description"},
    "d_items.csv": {"key": "itemid", "merge_file": ["chartevents.csv", "procedureevents.csv", "inputevents.csv"], "merge_column": "label"},
    "pharmacy.csv": {"key": "poe_id", "merge_file": "poe.csv", "merge_column": "medication"},
    "prescriptions.csv": {"key": "poe_id", "merge_file": "poe.csv", "merge_column": "drug"},
    "emar.csv": {"key": "poe_id", "merge_file": "poe.csv", "merge_column": "medication"},
}

EVENT_TYPE_MAP = {
            "transfers.csv": ("Transfer to Location", "intime"),
            "admissions.csv": [("Hospital Admission", "admittime"), ("Hospital Discharge", "dischtime")],
            "discharges.csv": ("Hospital Discharge", "dischtime"),
            "icustays.csv": [("ICU Admission", "intime"), ("ICU Discharge", "outtime")],
            "labevents.csv": ("Lab Test", "charttime"),
            "diagnoses_icd.csv": ("Diagnosis", None),
            "chartevents.csv": ("ICU Machine/Test Event", "charttime"),
            "procedureevents.csv": ("ICU Procedure", "starttime"),
            "inputevents.csv": ("ICU Nutrition/Drug Administration", "starttime"),
            "procedures_icd.csv": ("Hospital Procedure", "chartdate"),
            "hcpcsevents.csv": ("Hospital Service", "chartdate"),
            "poe.csv": ("order_type", "ordertime")  
        }

d_lookups = {}
for d_filename, d_info in tqdm(FILE_MERGE_MAP.items(), desc="Processing merge files"):
    d_path = os.path.join(base_dir, d_filename)
    
    if os.path.exists(d_path):
        with open(d_path, "r", encoding="utf-8") as f:
            reader = csv.DictReader(f)

            d_lookups[d_filename] = {
                row[d_info["key"]]: row.get(d_info["merge_column"], "UNKNOWN")  
                for row in reader if d_info["key"] in row  
            }


Processing merge files:   0%|          | 0/8 [00:00<?, ?it/s]

Processing merge files: 100%|██████████| 8/8 [04:04<00:00, 30.62s/it]


#### **Frontload**

In [7]:
patient_dict = defaultdict(list)

def process_csv(file_path, columns_to_keep, limit=None):
    
    with open(file_path, "r") as f:
        reader = csv.DictReader(f)
        for i, line in enumerate(tqdm(reader, desc=f"Processing {file_path}")):
            filtered_line = {key: line[key] for key in columns_to_keep if key in line}

            for d_filename, d_info in FILE_MERGE_MAP.items():
                if isinstance(d_info["merge_file"], list):
                    if any(file_path.endswith(f) for f in d_info["merge_file"]):
                        merge_key = filtered_line.get(d_info["key"])
                        if merge_key and merge_key in d_lookups.get(d_filename, {}):
                            filtered_line[d_info["merge_column"]] = d_lookups[d_filename][merge_key]
                elif file_path.endswith(d_info["merge_file"]):
                    merge_key = filtered_line.get(d_info["key"])
                    if merge_key and merge_key in d_lookups.get(d_filename, {}):
                        filtered_line[d_info["merge_column"]] = d_lookups[d_filename][merge_key]

            yield filtered_line

            if limit and i >= limit:
                break  


start_time = time.time()

for filename in tqdm(os.listdir(base_dir), desc="Processing CSV files"):
    if filename not in COLUMN_MAP:
        continue  

    file_path = os.path.join(base_dir, filename)
    columns_to_keep = COLUMN_MAP[filename]

    for row in process_csv(file_path, columns_to_keep, limit=3_500_000):
        patient_dict[row["subject_id"]].append(row)

elapsed_time = round(time.time() - start_time, 2)
print(f"Processed all CSV files in {elapsed_time} seconds with {len(patient_dict)} unique patients")

Processing D:\CourseworkFolder\DPSynthData\Data Manipulation\mimic-iv-3.1\extracted\admissions.csv: 546028it [00:03, 142297.76it/s]
Processing D:\CourseworkFolder\DPSynthData\Data Manipulation\mimic-iv-3.1\extracted\chartevents.csv: 3500000it [00:17, 195722.55it/s]
Processing D:\CourseworkFolder\DPSynthData\Data Manipulation\mimic-iv-3.1\extracted\diagnoses_icd.csv: 3500000it [00:13, 252226.76it/s]
Processing D:\CourseworkFolder\DPSynthData\Data Manipulation\mimic-iv-3.1\extracted\hcpcsevents.csv: 186074it [00:00, 241327.64it/s]
Processing D:\CourseworkFolder\DPSynthData\Data Manipulation\mimic-iv-3.1\extracted\icustays.csv: 94458it [00:00, 159971.88it/s]
Processing D:\CourseworkFolder\DPSynthData\Data Manipulation\mimic-iv-3.1\extracted\inputevents.csv: 3500000it [00:26, 134074.36it/s]
Processing D:\CourseworkFolder\DPSynthData\Data Manipulation\mimic-iv-3.1\extracted\labevents.csv: 3500000it [00:19, 181844.19it/s]
Processing D:\CourseworkFolder\DPSynthData\Data Manipulation\mimic-iv-

Processed all CSV files in 391.91 seconds with 364627 unique patients


#### **Event Classes**

In [8]:
class Event:
    def __init__(self, event_type, timestamp, details):
        self.event_type = event_type
        self.timestamp = timestamp
        self.details = details

    def format(self):
        for key in ["subject_id", "hadm_id", "poe_id", "event_id", "ordertime", "order_type"]:
            self.details.pop(key, None)
            
        if self.event_type == "Medications Ordered":
            self.details["medication_type"] = self.details.pop("event_subtype", None)

        if self.event_type not in ['Transfer to Location', 'Hospital Admission', 'Hospital Discharge', 'ICU Admission', 'ICU Discharge']:
            self.details.pop("location", None)

        self.details = {key: value for key, value in self.details.items() if value is not None}

    def __repr__(self):
        return f"{self.timestamp}: {self.event_type} - {self.details}"

#### **Stay Class**

In [9]:
class Stay:
    def __init__(self, subject_id, hadm_id):
        self.subject_id = subject_id
        self.hadm_id = hadm_id
        self.events = []


    def add_event(self, event):
        self.events.append(event)

    def get_timeline(self):
        return sorted(self.events, key=lambda e: (e.timestamp if e.timestamp is not None else float("inf")))

    def save_to_csv(self):
        df = pd.DataFrame([vars(event) for event in self.events])
        df.to_csv(f"timelines/{self.subject_id}_{self.hadm_id}_timeline.csv", index=False)

    def format(self):
        event_dict = defaultdict(lambda: {"count": 0, "event": None})
        new_events = []

        for event in self.events:
            if event.event_type == "ADT orders Ordered":
                continue  

            event_key = (event.event_type, event.timestamp, frozenset(event.details.items()))

            if event_key in event_dict:
                event_dict[event_key]["count"] += 1  
            else:
                event_dict[event_key] = {"count": 1, "event": event}

            if event.event_type in ["ICU Admission", "Hospital Admission"]:
                discharge_timestamp = event.details.get("dischtime") if event.event_type == "Hospital Admission" else event.details.get("outtime")
                
                if discharge_timestamp:
                    discharge_event = Event(
                        event_type="ICU Discharge" if event.event_type == "ICU Admission" else "Hospital Discharge",
                        timestamp=discharge_timestamp,
                        details={k: v for k, v in event.details.items() if k != "timestamp"} 
                    )
                    new_events.append(discharge_event)  

        unique_events = []
        for key, value in event_dict.items():
            event = value["event"]
            count = value["count"]
            if count > 1:
                event.event_type = f"{event.event_type} (x{count})"
            unique_events.append(event)

        self.events = unique_events + new_events  
        
    def parse_timestamp(self, timestamp):
        if not timestamp or timestamp in ["", "None"]:
            return datetime.max  
        try:
            return datetime.fromisoformat(timestamp) 
        except ValueError:
            return datetime.max 

    def sort(self):
        self.events.sort(key=lambda e: self.parse_timestamp(e.timestamp))
    

#### **Patient Class**

In [10]:
class Patient:
    def __init__(self, subject_id):
        self.subject_id = subject_id
        self.stays = {}

    def process_events(self):
        if self.subject_id not in patient_dict:
            return 

        for row in patient_dict[self.subject_id]:
            hadm_id = row.get("hadm_id")
            if not hadm_id:
                continue 

            if hadm_id not in self.stays:
                self.stays[hadm_id] = Stay(self.subject_id, hadm_id)

            source_file = self.get_source_file(row)
            if source_file not in EVENT_TYPE_MAP:
                event_type, timestamp_key = "Unknown Event", None
            else:
                event_type, timestamp_key = EVENT_TYPE_MAP[source_file]

            if isinstance(event_type, list):
                for etype, ts_key in event_type:
                    timestamp = row.get(ts_key)
                    event = Event(etype, timestamp, row)
                    self.stays[hadm_id].add_event(event)
                continue 

            if source_file == "poe.csv":
                event_type = row.get("order_type", "Unknown Order") + " Ordered"

            timestamp = row.get(timestamp_key)
            event = Event(event_type, timestamp, row)
            event.format()
            self.stays[hadm_id].add_event(event)

    def get_source_file(self, row):
        for file, columns in COLUMN_MAP.items():
            if all(key in row for key in columns):  
                return file
        return "unknown.csv"  

    def process_stays(self):
        for stay in self.stays.values():
            stay.sort()
            stay.format()

    def save_all_timelines(self):
        for stay in self.stays.values():
            stay.save_to_csv()


#### **Update Cache**

In [11]:
def update_cache(processed_patients, cache_file="patient_cache.json"):
    temp_file = cache_file + ".tmp"
    try:
        with open(temp_file, "w") as f:
            json.dump(list(processed_patients), f)  
        os.replace(temp_file, cache_file)  
    except Exception as e:
        print(f"Failed to update cache: {e}")

#### **Main**

In [12]:
base_dir = r"D:\CourseworkFolder\DPSynthData\Data Manipulation\mimic-iv-3.1"

if not os.path.exists("timelines"):
    os.makedirs("timelines")

cache_file = "patient_cache.json"
if os.path.exists(cache_file):
    with open(cache_file, "r") as f:
        processed_patients = set(json.load(f))
else:
    processed_patients = set()

patient_list = patient_dict.keys()

for subject_id in tqdm(patient_list, desc="Processing Patients"):
    if subject_id in processed_patients:
        print(f"Patient {subject_id} already in cache - skipping")
        continue  

    start_time = time.time()
    patient = Patient(subject_id)
    
    patient.process_events() 
    patient.process_stays()
    patient.save_all_timelines()
    
    elapsed_time = round(time.time() - start_time, 2)
    print(f"Processed Patient {subject_id} in {elapsed_time} seconds")

    processed_patients.add(subject_id)
    update_cache(processed_patients)


Processing Patients:   0%|          | 0/364627 [00:00<?, ?it/s]

Patient 10000032 already in cache - skipping
Patient 10000068 already in cache - skipping
Patient 10000084 already in cache - skipping
Patient 10000108 already in cache - skipping
Patient 10000117 already in cache - skipping
Patient 10000161 already in cache - skipping
Patient 10000248 already in cache - skipping
Patient 10000280 already in cache - skipping
Patient 10000560 already in cache - skipping
Patient 10000635 already in cache - skipping
Patient 10000690 already in cache - skipping
Patient 10000719 already in cache - skipping
Patient 10000764 already in cache - skipping
Patient 10000826 already in cache - skipping
Patient 10000883 already in cache - skipping
Patient 10000886 already in cache - skipping
Patient 10000904 already in cache - skipping
Patient 10000935 already in cache - skipping
Patient 10000947 already in cache - skipping
Patient 10000969 already in cache - skipping
Patient 10000980 already in cache - skipping
Patient 10001176 already in cache - skipping
Patient 10

Processing Patients:   7%|▋         | 26043/364627 [00:00<00:01, 259520.07it/s]

Patient 10954107 already in cache - skipping
Patient 10954117 already in cache - skipping
Patient 10954146 already in cache - skipping
Patient 10954257 already in cache - skipping
Patient 10954285 already in cache - skipping
Patient 10954392 already in cache - skipping
Patient 10954424 already in cache - skipping
Patient 10954441 already in cache - skipping
Patient 10954488 already in cache - skipping
Patient 10954531 already in cache - skipping
Patient 10954543 already in cache - skipping
Patient 10954581 already in cache - skipping
Patient 10954647 already in cache - skipping
Patient 10954681 already in cache - skipping
Patient 10954764 already in cache - skipping
Patient 10954823 already in cache - skipping
Patient 10954917 already in cache - skipping
Patient 10955000 already in cache - skipping
Patient 10955059 already in cache - skipping
Patient 10955095 already in cache - skipping
Patient 10955123 already in cache - skipping
Patient 10955169 already in cache - skipping
Patient 10

Processing Patients:  12%|█▏        | 43792/364627 [00:12<01:50, 2892.35it/s]  

...

Processing Patients:  15%|█▍        | 53463/364627 [05:09<3:10:49, 27.18it/s]

Processed Patient 12406838 in 0.0 seconds
Processed Patient 12406866 in 0.0 seconds
Processed Patient 12406896 in 0.02 seconds
Processed Patient 12406991 in 0.0 seconds
Processed Patient 12407111 in 0.0 seconds
Processed Patient 12407162 in 0.02 seconds


Processing Patients:  15%|█▍        | 53470/364627 [05:09<30:03, 172.54it/s] 


Processed Patient 12407276 in 0.0 seconds
Processed Patient 12407300 in 0.0 seconds
Processed Patient 12407328 in 0.01 seconds
Processed Patient 12407382 in 0.0 seconds
Processed Patient 12407385 in 0.0 seconds


KeyboardInterrupt: 